# 빅분기 작업형2 Baseline
- 결과창이 시각적으로 보기 좋지 않으니 print()에 `, "\n"` 넣어주기
- train, test 데이터에서 범주형 변수의 nunique 값이 다를 때?
    - 두 데이터를 먼저 합친 후 인코딩 진행, 그리고 다시 데이터 분리

- 모델 디폴트 값
    - 랜덤포레스트 : max_depth= None, n_estimators = 100
    - lgbm : max_depth=-1, n_estimators = 100 

In [ ]:
import pandas as pd
import numpy as np

# 0. 데이터 로드
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")

print("train=======================")
print(train.shape)
print(train.head(), "\n")

print("test=======================")
print(test.shape)
print(test.head(), "\n")



# 1. EDA
# 1-1) train EDA
print(train.info(), "\n")
print(train.isnull().sum(), "\n")
print(train.describe(), "\n")

# 1-2) test EDA
print(test.info(), "\n")
print(test.isnull().sum(), "\n")
print(test.describe(), "\n")

# 범주형 칼럼 고유값 비교
print(train.describe(include="O"), "\n")
print(test.describe(include="O"), "\n")



# 2. 전처리
# 2-1) 결측치 처리, 불필요 칼럼 제거, X와 y 분리

# 2-1-1)중앙값으로 결측치 대체
train["결측 존재 칼럼"] = train["결측 존재 칼럼"].fillna(train["결측 존재 칼럼"].median()) 
test["결측 존재 칼럼"] = test["결측 존재 칼럼"].fillna(test["결측 존재 칼럼"].median())

# 결측치 채워졌나 확인
print(train.isnull().sum(), "\n")
print(test.isnull().sum(), "\n")

# 2-1-2) id 같은 불필요 칼럼 제거
train = train.drop("id", axis=1)

test_id = test.pop("id") # 결과 제출할 때 test_id 필요할 수 있음

# 2-1-3) X와 y분리
y = train.pop("예측할 타겟 칼럼")

print("train=======================")
print(train.head(3), "\n")
print("test=======================")
print(test.head(3), "\n")
print(y.head(1), "\n")



# 2-2) 스케일링 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# 연속형 칼럼 선택
print(train.info(), "\n")
con_cols = train.select_dtypes(exclude="object").copy().columns
print("연속형 칼럼 : ", con_cols)

train[con_cols] = scaler.fit_transform(train[con_cols])
test[con_cols] = scaler.transform(test[con_cols])



# 2-3) 인코딩 
# 원핫인코딩 : train & test nunique 동일하고 
# nunique 개수가 많지 않다고 판단 될 떄
train = pd.get_dummies(train)
test = pd.get_dummies(test)

test = test[train.columns] # 칼럼 순서 맞춰주기



# 라벨인코딩 : train, test nunique 다를 때 & 범주형 데이터 nunique 많을 때
# 먼저 train, test 합쳐주기
df = pd.concat([train, test])

from sklearn.preprocessing import LabelEncoder

# 범주형 칼럼 선택
cat_cols = train.select_dtypes(include="object").copy().columns
print("범주형 칼럼 : ", cat_cols )

for col in cat_cols : 
    le = LabelEncoder()
    df[col] = le.fit_transfrom(df[col])

# train, test 분리
train = df[: train.shape[0]].copy()
test = df[train.shape[0]:].copy()



# 스케일링, 인코딩 잘 적용됐나 확인
print("train=======================")
print(train.head(3), "\n")
print("test=======================")
print(test.head(3), "\n")



# 3. 검증 데이터 분리
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, 
                                                  y, 
                                                  random_state=1, 
                                                  stratify=y) # 분류일 때만 !! 

print(X_train.shape, X_val.shape, "\n")



# 4. 모델링 : 랜덤포레스트, lgbm 2개 준비
# 4-1) 성능지표
# 분류 성능지표
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from sklearn.metrics import classification_report

# 회귀 성능지표
from sklearn.metrics import mean_squared_error, r2_score



# 4-2) 분류 모델
# 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=1)
rf.fit(X_train, y_train)

rf_pred = rf.predict(X_val) 
rf_pred_proba = rf.predict_proba(X_val)[:, 1]

# roc_auc_score
rf_roc = roc_auc_score(y_val, rf_pred_proba)

# f1_score
rf_f1 = f1_score(y_val, rf_pred, 
                 average = "macro") # 다중분류일 때

# accuracy_score
rf_acu = accuracy_score(y_val, rf_pred)



# lgbm
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(random_state=1)
lgbm.fit(X_train, y_train)

lgbm_pred = lgbm.predict(X_val) # 클래스에 대한 예측값
lgbm_pred_proba = lgbm.predict_proba(X_val)[:, 1] # 클래스 1에 대한 예측 '확률'

# roc_auc_score
lgbm_roc = roc_auc_score(y_val, lgbm_pred_proba)

# f1_score
lgbm_f1 = f1_score(y_val, lgbm_pred,
                   average = "macro") # 다중분류일 때

# accuracy_score
lgbm_acu = accuracy_score(y_val, lgbm_pred)



# 4-3) 회귀 모델
# 랜덤포레스트
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=1)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_val)

# rmse
rf_rmse = np.sqrt(mean_squared_error(y_val, rf_pred))

# r2_score
rf_r2 = r2_score(y_val, rf_pred)



# lgbm
from lightgbm import LGBMRegressor
lgbm = LGBMRegressor(random_state=1)
lgbm.fit(X_train, y_train)
lgbm_pred = lgbm.predict(X_val)

# rmse
lgbm_rmse = np.sqrt(mean_squared_error(y_val, lgbm_pred))

# r2_score
lgbm_r2 = r2_score(y_val, lgbm_pred)



# 최종 모델 선택하여 test 데이터에 대해 pred 지정
# 분류일 때 예시
# 클래스 예측 & 회귀도 동일
pred = lgbm.predict(test)

# 클래스 예측 확률
# 정확히 클래스가 어떤 경우의 확률을 말하는지 문제 꼼꼼히 읽을 것 
pred = lgbm.predict_proba(test)[:, 1] 



# 5. 제출 : df, csv
submit = pd.DataFrame({"id" : test_id, "pred / 제출 칼럼명 자세히 보기" : pred})
submit.to_csv("수험번호.csv", index=False)

check = pd.read_csv("수험번호.csv") # 제출 잘 됐는지 확인
print(check.head(3))

## 하이퍼 파라미터 튜닝 : 그리드 서치
### 분류
| 성능 지표  | scoring 옵션   |
|---|---|
| roc_auc_score  | scoring = "roc_auc"   |
| f1_score  |  scoring = "f1" |
| macro f1_score  |        scoring = scorer              |

```python 
# 다중분류 일 때 scorer 생성 후, scoring = scorer 할당
scorer = make_scorer(f1_score, average='macro')
```

In [ ]:
# 분류 하이퍼 파라미터 튜닝
from sklearn.metrics import make_scorer

from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=2023)

from lightgbm import LGBMClassifier
lgbm = LGBMClassifier

models = [rf, lgbm]

params= {"max_depth" : [1,2,3], "n_estimators" : [100, 200, 300]}

# # 다중분류, macro f1 일 때
# socrer = make_scorer(f1_score, average="macro")

# scoring="f1" / scoring = scorer

for model in models :
    gs = GridSearchCV(model, cv=5, param_grid = params, scoring="roc_auc", n_jobs=4) 
    gs.fit(X_train, y_train)
    
    print("=="10)
    print(f"model : {model}")
    print(f"최적 파라미터 : {gs.best_params_}")
    print(f"best score : {gs.best_score_}")

### 회귀
| 성능 지표  | scoring 옵션   |
|---|---|
| r2_score  | scoring = "r2"   |
| RMSE  |  scoring="neg_root_mean_squared_error" |
| mean_absolute_error |       scoring="neg_mean_absolute_error"        |

- 회귀의 지표 RMSE, MAE는 작을수록 성능이 좋기 때문에, -gs.best_score_ 이렇게 점수에 마이너스 붙여줘야 함 !


In [ ]:
# 회귀 하이퍼 파라미터 튜닝
from sklearn.metrics import mean_squared_error, make_scorer, r2_score

from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=2023)

from lightgbm import LGBMRegressor
lgbm = LGBMRegressor(random_state=2023)

models = [rf, lgbm]
params = {"max_depth" : [1, 2, 3], "n_estimators" : [100, 200, 300]}

for model in models :
    gs = GridSearchCV(model, cv=5, param_grid=params, 
                      scoring="neg_root_mean_squared_error", n_jobs=4)
    gs.fit(X_train, y_train)
    
    print("=="*10)
    print(f"model : {model}")
    print(f"최적 파라미터 : {gs.best_params_}")
    print(f"최적 점수 : {-gs.best_score_}")

### 교차 검증은 모델의 성능을 신뢰할 수 있는 추정치를 제공하기 위해 사용되지만, 실제 검증 데이터에서의 성능은 교차 검증의 best_score와 차이가 있을 수 있다. 왜 그럴까?

$\rightarrow$ 최적화된 파라미터가 학습 데이터에 과적합(overfitting)되어 일반화 능력이 저하될 수 있기 떄문 !
- 그리드 서치는 학습 데이터를 기반으로 모델의 파라미터 조합을 탐색하여 최적의 조합을 탐색
- 그러나 학습 데이터에 대해서는 최적의 조합일지라도, 검증용 데이터나 새로운 데이터에 대해서는 일반화 성능이 좋지 않을 수 있음
- 모델이 학습 데이터에 지나치게 맞추어져서 새로운 패턴이나 변동성을 제대로 반영하지 못하는 경우가 있기 때문

